In [1]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession


keyfile_path = "config/btcanalysishust-495a3a227f22.json"

spark = SparkSession.builder \
    .appName("KafkaConsumer") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.0,com.google.cloud.bigdataoss:gcs-connector:hadoop3-2.2.5,com.google.guava:guava:30.0-jre")\
    .config("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .config("spark.hadoop.fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem") \
    .config("spark.hadoop.fs.gs.auth.type", "OAuth2") \
    .config("spark.hadoop.fs.gs.project.id", "btcanalysishust") \
    .config("spark.hadoop.fs.gs.input.close.input.streams.after.task.complete", "true") \
    .config("spark.hadoop.google.cloud.auth.service.account.json.keyfile", keyfile_path) \
    .getOrCreate()



24/11/25 09:08:37 WARN Utils: Your hostname, MacBook-Pro-cua-My.local resolves to a loopback address: 127.0.0.1; using 10.17.42.126 instead (on interface en0)
24/11/25 09:08:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/Users/ibulmnie/Documents/20241/BigData/crypto-big-data/spark-env/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/ibulmnie/.ivy2/cache
The jars for the packages stored in: /Users/ibulmnie/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
com.google.cloud.bigdataoss#gcs-connector added as a dependency
com.google.guava#guava added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e173977c-38b2-4ebc-be60-3d436e15309e;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.2.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.2.0 in central
	found org.apache.kafka#kafka-clients;2.8.0 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.1 in central
	found org.apache.htrace#htrace-core4;4.1.0-incubating in cent

In [ ]:
# Đọc dữ liệu từ Kafka
df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "35.206.252.44:9092") \
    .option("subscribe", "crypto-prices") \
    .option("startingOffsets", "earliest") \
    .load()

# Kafka data có các cột "key" và "value" dưới dạng binary
df = df.selectExpr("CAST(value AS STRING) as value")

from pyspark.sql.types import StructType, StructField, StringType, FloatType

schema_log = StructType([
    StructField("timestamp", StringType()),
    StructField("prices", StructType([
        StructField("bitcoin", FloatType()),
        StructField("ethereum", FloatType()),
        StructField("tether", FloatType()),
        StructField("binancecoin", FloatType()),
        StructField("usd-coin", FloatType()),
        StructField("ripple", FloatType()),
        StructField("cardano", FloatType()),
        StructField("dogecoin", FloatType()),
        StructField("matic-network", FloatType()),
        StructField("solana", FloatType()),        
    ]))
])

from pyspark.sql.functions import col, from_json, explode, udf

crypto_parsed_df = df.select(from_json(col("value"), schema_log).alias("data"))


crypto_parsed_df = crypto_parsed_df.select(
    col("data.timestamp").alias("timestamp"),
    col("data.prices.bitcoin").alias("BTC"),
    col("data.prices.ethereum").alias("ETH"),
    col("data.prices.tether").alias("TETHER"),
    col("data.prices.binancecoin").alias("BNB"),
    col("data.prices.usd-coin").alias("USDC"),
    col("data.prices.ripple").alias("XRP"),
    col("data.prices.cardano").alias("ADA"),
    col("data.prices.dogecoin").alias("DOGE"),
    col("data.prices.matic-network").alias("MATIC"),
    col("data.prices.solana").alias("SOLANA")
)


# Hiển thị dữ liệu đọc được ra console (cho thử nghiệm), không cắt ngắn các giá trị
crypto_parsed_df.writeStream.format("console").option("truncate", "false").start().awaitTermination()


In [2]:
df = spark.read.parquet("gs://crypto-historical-data-2/ver1/BTC/2024/01/data.parquet")
df.show()


24/11/25 09:08:52 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: gs://crypto-historical-data-2/ver1/BTC/2024/01/data.parquet.
java.lang.RuntimeException: java.lang.reflect.InvocationTargetException
	at org.apache.hadoop.util.ReflectionUtils.newInstance(ReflectionUtils.java:137)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3467)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.streaming.FileStreamSink$.hasMetadata(FileStreamSink.scala:53)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:366)
	at org.apache.spark.sql.DataFrameReader.loadV1Source(DataFr

Py4JJavaError: An error occurred while calling o42.parquet.
: java.lang.RuntimeException: java.lang.reflect.InvocationTargetException
	at org.apache.hadoop.util.ReflectionUtils.newInstance(ReflectionUtils.java:137)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3467)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.datasources.DataSource$.$anonfun$checkAndGlobPathIfNecessary$1(DataSource.scala:724)
	at scala.collection.immutable.List.map(List.scala:293)
	at org.apache.spark.sql.execution.datasources.DataSource$.checkAndGlobPathIfNecessary(DataSource.scala:722)
	at org.apache.spark.sql.execution.datasources.DataSource.checkAndGlobPathIfNecessary(DataSou

24/11/25 09:08:58 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
